In [230]:
from pydantic import BaseModel
from typing import List


class Car(BaseModel):
    brand: str


class Cars(BaseModel):
    __root__: List[Car] = []

In [231]:
# assign cars during instance creation
cars = Cars(__root__=[Car(brand="Audi"), Car(brand="BMW")])
cars

Cars(__root__=[Car(brand='Audi'), Car(brand='BMW')])

In [235]:
# we get valid JSON
cars.json()

'[{"brand": "Audi"}, {"brand": "BMW"}]'

In [233]:
# assign cars after instance creation
cars = Cars()
cars.__root__=[Car(brand="Audi"), Car(brand="BMW")]
cars

Cars(__root__=[Car(brand='Audi'), Car(brand='BMW')])

In [178]:
from pydantic import BaseModel
from typing import List


class Passenger(BaseModel):
    name: str
    age: int


class Car(BaseModel):
    brand: str
    passengers: List[Passenger] = []

In [179]:
car = Car(
    brand="Audi",
    passengers=[
        Passenger(name="Hans", age=21),
        Passenger(name="Martin", age=23),
    ])
car

Car(brand='Audi', passengers=[Passenger(name='Hans', age=21), Passenger(name='Martin', age=23)])

In [180]:
car.json()

'{"brand": "Audi", "passengers": [{"name": "Hans", "age": 21}, {"name": "Martin", "age": 23}]}'

In [181]:
empty_car = Car(
    brand="Audi",
)
empty_car

Car(brand='Audi', passengers=[])

In [182]:
empty_car.json()

'{"brand": "Audi", "passengers": []}'

In [183]:
car.passengers.json()

AttributeError: 'list' object has no attribute 'json'

In [236]:
from pydantic import BaseModel
from typing import List


class Passenger(BaseModel):
    name: str
    age: int


class Passengers(BaseModel):
    __root__: List[Passenger] = []


class Car(BaseModel):
    brand: str
    passengers: Passengers

In [237]:
car = Car(
    brand="Audi",
    passengers=[
        Passenger(name="Hans", age=21),
        Passenger(name="Martin", age=23),
    ])
car

Car(brand='Audi', passengers=Passengers(__root__=[Passenger(name='Hans', age=21), Passenger(name='Martin', age=23)]))

In [238]:
car.json()

'{"brand": "Audi", "passengers": [{"name": "Hans", "age": 21}, {"name": "Martin", "age": 23}]}'

In [239]:
car.passengers.json()

'[{"name": "Hans", "age": 21}, {"name": "Martin", "age": 23}]'

In [240]:
empty_car = Car(
    brand="Audi",
)
empty_car

ValidationError: 1 validation error for Car
passengers
  field required (type=value_error.missing)

In [196]:
empty_car.json()

'{"brand": "Audi", "passengers": []}'

In [241]:
empty_car.passengers.json()

AttributeError: 'list' object has no attribute 'json'

In [149]:
# simplyfied production code of entity aggregate root (entity which acts as aggregate root), entities and value objects
from pydantic import BaseModel

from typing import List, Optional


class ValueObject(BaseModel):
    ...

    class Config:
        frozen = True


class Entity(BaseModel):
    ...


class AggregateRoot(Entity):
    ...


class Order(AggregateRoot):
    id: int
    order_lines: 'OrderLines'
    overall_price: int = 0

    def add_order_line(self, order_line: 'OrderLine') -> None:
        self.order_lines.append(order_line)
        self.overall_price += order_line.product_count * order_line.product.price


class Product(Entity):
    product_number: int
    price: int


class OrderLines(BaseModel):
    __root__: List['OrderLine']


class OrderLine(ValueObject):
    product: 'Product'
    product_count: int

In [150]:
# production code for repository
import sqlite3

from typing import Protocol


# option A: The "I prefer composition over inheritance" way of defining an interface for repositories (typing without inheritance).
class OrderRepository(Protocol):
    """Defines the interface (methods) the concrete repository of the order entity needs to satisfy."""
    def add(self, order: Order) -> None:
        """Adds an order to persisted orders."""

    def get(self, order_id: int) -> Order:
        """Gets an order from persisted orders."""

    def remove(self, order_id: int) -> None:
        """Removes an order from persisted orders."""


class FakeProtocolOrderRepository:
    """Fake implementation of a concrete repository (using protocol) which uses memors for perstisting orders."""
    def __init__(self):
        self._orders = []

    def add(self, order: Order) -> None:
        self._orders.append(order)

    def get(self, order_id: int) -> Order:
        for order in self._orders:
            if order_id == order.id:
                return order

    def remove(self, order_id: int) -> None:
        for order in self._orders:
            if order_id == order.id:
                self._orders.remove(order)
                return


class SqliteProtocolOrderRepository:
    """Implementation of a concrete repository (using protocol) which uses SQLite database for persisting orders."""
    def __init__(self, con: sqlite3.Connection = sqlite3.connect("file::memory:?cache=shared")):
        self._con = con

    def setup(self):
        self._con.execute("""
            CREATE TABLE "order" (
            "id" INTEGER NOT NULL UNIQUE,
            "order_lines" TEXT NOT NULL,
            "overall_price" INTEGER NOT NULL,
            PRIMARY KEY("id")
            );
        """)
        self._con.commit()
        self._con.close()

    def add(self, order: Order) -> None:
        self._con.execute(f"INSERT INTO order (id, order_lines, overall_price) VALUES({order.id}, {order.order_lines.json()}, {order.overall_price});")
        self._con.commit()
        self._con.close()

    def get(self, order_id: int) -> Order:
        self._con.execute(f"SELECT * FROM order WHERE id={order_id};")
        self._con.close()

    def remove(self, order_id: int) -> None:
        self._con.execute(f"DELETE FROM order WHERE id={order_id};")
        self._con.commit()
        self._con.close()


In [151]:
# option B: The "I prefer inheritance over composition" way of defining an interface for repositories (inheriting, no typing).
import abc


class AbstractOrderRepository(abc.ABC):
    """Defines an abstract base class the concrete repository of the order entity needs to inherit from."""
    @abc.abstractmethod
    def add(self, order: Order) -> None:
        """Adds an order to persisted orders."""
        raise NotImplementedError

    @abc.abstractmethod
    def get(self, order_id: int) -> Order:
        """Gets an order from persisted orders."""
        raise NotImplementedError

    @abc.abstractmethod
    def remove(self, order_id: int) -> None:
        """Removes an order from persisted orders."""
        raise NotImplementedError


class FakeAbcOrderRepository(AbstractOrderRepository):
    """Fake implementation of a concrete repository which uses memory for perstisting orders."""

    def __init__(self):
        self._orders = []

    def add(self, order: Order) -> None:
        self._orders.append(order)

    def get(self, order_id: int) -> Order:
        return next(o for o in self._orders if o.order_id == order_id)

    def remove(self, order_id: int) -> None:
        for order in self._orders:
            if order_id == order.order_id:
                self._orders.remove(order)
                return
        return

- 1 repository per 1 aggregate root <-> 1 aggregate root per 1 repository
- In CRUD apps the model depends on ORM
- Repository inverts the dependency (dependency inversion) between persistence layer and model. The persistence layer (ORM) depends on the model. Instead of: Model depends on the persistence layer.
- Repository is abstraction of premanent storage.

In [152]:
order_1 = Order(id=1)
order_1.add_order_line(OrderLine(product=Product(product_number=11, price=100), product_count=1))
order_1.add_order_line(OrderLine(product=Product(product_number=12, price=200), product_count=2))
order_1

AttributeError: 'NoneType' object has no attribute 'append'

In [138]:
order_2 = Order(id=2)
order_2.add_order_line(OrderLine(product=Product(product_number=21, price=300), product_count=3))
order_2.add_order_line(OrderLine(product=Product(product_number=22, price=400), product_count=4))
order_2

Order(id=2, order_lines=[OrderLine(product=Product(product_number=21, price=300), product_count=3), OrderLine(product=Product(product_number=22, price=400), product_count=4)], overall_price=2500)

In [139]:
fake_protocol_permanent_storage = FakeProtocolOrderRepository()
fake_protocol_permanent_storage.add(order_1)
fake_protocol_permanent_storage.add(order_2)
fake_protocol_permanent_storage

In [140]:
order_1_from_permanent_storage = fake_protocol_permanent_storage.get(1)
order_1_from_permanent_storage

Order(id=1, order_lines=[OrderLine(product=Product(product_number=11, price=100), product_count=1), OrderLine(product=Product(product_number=12, price=200), product_count=2)], overall_price=500)

In [141]:
order_2_from_permanent_storage = fake_protocol_permanent_storage.get(2)
order_2_from_permanent_storage

Order(id=2, order_lines=[OrderLine(product=Product(product_number=21, price=300), product_count=3), OrderLine(product=Product(product_number=22, price=400), product_count=4)], overall_price=2500)

In [142]:
order_repo = SqliteProtocolOrderRepository()

In [143]:
order_repo.setup()

OperationalError: table "order" already exists

In [144]:
order_repo.add(order_1)

AttributeError: 'list' object has no attribute 'json'

In [82]:
order_1.json()

'{"id": 1, "order_lines": [{"product": {"product_number": 11, "price": 100}, "product_count": 1}, {"product": {"product_number": 12, "price": 200}, "product_count": 2}], "overall_price": 500}'

In [77]:
order_1.order_lines.json()

AttributeError: 'list' object has no attribute 'json'

In [78]:
order_1.order_lines

[OrderLine(product=Product(product_number=11, price=100), product_count=1),
 OrderLine(product=Product(product_number=12, price=200), product_count=2)]

In [79]:
orders_jsonifiable = []
for order in order_1.order_lines:
    orders_jsonifiable.append(order.json())
orders_jsonifiable

['{"product": {"product_number": 11, "price": 100}, "product_count": 1}',
 '{"product": {"product_number": 12, "price": 200}, "product_count": 2}']

In [81]:
json.dumps(orders_jsonifiable)

'["{\\"product\\": {\\"product_number\\": 11, \\"price\\": 100}, \\"product_count\\": 1}", "{\\"product\\": {\\"product_number\\": 12, \\"price\\": 200}, \\"product_count\\": 2}"]'